In [6]:
import os
import numpy as np
from tqdm import tqdm
import mne
from scipy.signal import stft
from skimage.transform import resize

# ---------------------------
# Labeling function
# ---------------------------
def get_label(s):
    if 1 <= s <= 36:
        return 2  # AD
    elif 37 <= s <= 65:
        return 0  # HC
    elif 66 <= s <= 88:
        return 1  # FTD
    else:
        return None

# ---------------------------
# Brain lobe mapping
# ---------------------------
BRAIN_LOBES = {
    "frontal":   ["Fp1","Fp2","F7","F3","Fz","F4","F8"],
    "central":   ["C3","Cz","C4"],
    "temporal":  ["T3","T4","T5","T6"],
    "parietal":  ["P3","P4","Pz"],
    "occipital": ["O1","O2","Oz"],
    "full": None
}

# ---------------------------
# STFT to spectrogram
# ---------------------------
def compute_spectrogram(signal, fs, size=224):
    f, t, Zxx = stft(signal, fs=fs, nperseg=256, noverlap=128, window="hamming")
    spec = np.abs(Zxx)
    # normalize
    spec = (spec - spec.min()) / (spec.max() - spec.min() + 1e-8)
    # resize to 224x224
    return resize(spec, (size, size), anti_aliasing=True).astype(np.float32)

# ---------------------------
# Main
# ---------------------------
def main():
    INPUT_DIR = r"C:\Users\fathi\Desktop\Rest_eeg_ds004504-download\derivatives"
    SAVE_DIR  = r"C:\Users\fathi\Desktop\EEG_spectrograms_npy"
    os.makedirs(SAVE_DIR, exist_ok=True)

    FS_ORIG = 500
    FS_TARGET = 256   # paper resample

    SEGMENTS = {
        "5s":   5,
        "10s":  10,
        "15s":  15,
        "20s":  20,
        "25s":  25,
        "30s":  30,
    }

    for sub_folder in tqdm(os.listdir(INPUT_DIR), desc="Subjects"):
        if not sub_folder.startswith("sub-"):
            continue

        subj_id = int(sub_folder.split("-")[1])
        label_val = get_label(subj_id)
        if label_val is None:
            continue

        eeg_path = os.path.join(INPUT_DIR, sub_folder, "eeg", f"{sub_folder}_task-eyesclosed_eeg.set")
        if not os.path.exists(eeg_path):
            continue

        raw = mne.io.read_raw_eeglab(eeg_path, preload=True, verbose=False)
        raw.resample(FS_TARGET)  # resample to match paper

        # Collect boundary annotations
        boundaries = [
            (int(ann["onset"] * FS_TARGET), int((ann["onset"] + ann["duration"]) * FS_TARGET))
            for ann in raw.annotations if ann["description"].lower() == "boundary"
        ]

        data = raw.get_data()
        ch_names = raw.ch_names
        total_len = data.shape[1]

        for seg_name, seg_sec in SEGMENTS.items():
            seg_len = seg_sec * FS_TARGET
            stride = seg_len  # no overlap
            num_segments = (total_len - seg_len) // stride + 1

            for i in range(num_segments):
                start, end = i * stride, i * stride + seg_len

                # skip if segment overlaps boundary
                if any((start < b_end and end > b_start) for b_start, b_end in boundaries):
                    continue

                for lobe_name, channels in BRAIN_LOBES.items():
                    if channels is None:
                        seg_data = data[:, start:end].mean(axis=0)
                    else:
                        idx = [ch_names.index(ch) for ch in channels if ch in ch_names]
                        if not idx: 
                            continue
                        seg_data = data[idx, start:end].mean(axis=0)

                    spec = compute_spectrogram(seg_data, FS_TARGET)

                    save_path = os.path.join(SAVE_DIR, seg_name, lobe_name, str(label_val))
                    os.makedirs(save_path, exist_ok=True)
                    out_file = os.path.join(save_path, f"{sub_folder}_seg{i}.npy")
                    np.save(out_file, spec)

    print("✅ Finished saving spectrograms for all segment lengths (paper-style).")

if __name__ == "__main__":
    main()


Subjects:   2%|▏         | 2/88 [00:37<27:36, 19.26s/it]C:\Users\fathi\AppData\Local\Temp\ipykernel_16712\2241562197.py:77: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(eeg_path, preload=True, verbose=False)
Subjects:   3%|▎         | 3/88 [00:43<19:16, 13.60s/it]C:\Users\fathi\AppData\Local\Temp\ipykernel_16712\2241562197.py:77: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(eeg_path, preload=True, verbose=False)
Subjects:   6%|▌         | 5/88 [01:16<21:04, 15.24s/it]C:\Users\fathi\AppData\Local\Temp\ipykernel_16712\2241562197.py:77: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(eeg_path, preload=True

✅ Finished saving spectrograms for all segment lengths (paper-style).


network

In [ ]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

class SpectrogramDataset(Dataset):
    def __init__(self, root_dir, segment="5s", lobe="full", transform=None):
        """
        root_dir: where spectrograms are saved
        segment: which window length (e.g. "5s", "10s")
        lobe: which brain lobe (e.g. "frontal", "full")
        """
        self.samples = []
        self.transform = transform

        path = os.path.join(root_dir, segment, lobe)
        for label in os.listdir(path):
            class_path = os.path.join(path, label)
            if not os.path.isdir(class_path):
                continue
            for file in os.listdir(class_path):
                if file.endswith(".npy"):
                    self.samples.append((os.path.join(class_path, file), int(label)))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        file_path, label = self.samples[idx]
        spec = np.load(file_path)  # (224, 224)
        spec = np.expand_dims(spec, axis=0)  # (1, 224, 224) for CNN

        if self.transform:
            spec = self.transform(spec)

        return torch.tensor(spec, dtype=torch.float32), torch.tensor(label, dtype=torch.long)


In [ ]:
"""
EEG Spectrogram 5-Fold CV Trainer
---------------------------------
- Loads ALL segment lengths and ALL brain lobes found under a given root dir.
- Runs two binary experiments: (0 vs 1) and (0 vs 2).
- Stratified 5-fold CV with class-weighted loss.
- Tracks per-epoch validation metrics (accuracy, precision, recall, F1).
- Selects the BEST EPOCH per fold by **macro F1** and saves its checkpoint.
- Aggregates results across folds, lobes, and segments; finds the best segment length and best lobe by F1.
- Saves:
    * Per-fold best model checkpoints
    * Per-epoch logs per fold (CSV)
    * A global summary CSV of all combinations and folds
    * A topline CSV highlighting the best segment length and best lobe for each experiment

Directory layout expected (created by your generation script):
root_dir/
  ├── 5s/
  │    ├── frontal/0/*.npy, 1/*.npy, 2/*.npy
  │    ├── central/...
  │    ├── ...
  │    └── full/...
  ├── 10s/
  │    └── ...
  └── ...

Run:
    python eeg_cv_training_pipeline.py \
        --root_dir "C:/Users/fathi/Desktop/EEG_spectrograms_npy" \
        --results_dir "C:/Users/fathi/Desktop/eeg_cv_results" \
        --epochs 20 --batch_size 32 --lr 1e-3

"""

import os
import json
import math
import time
import random
import argparse
from pathlib import Path

import numpy as np
import pandas as pd
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, Subset

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import (
    accuracy_score,
    precision_recall_fscore_support,
)

# -----------------------------
# Reproducibility
# -----------------------------

def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

root_dir = r"C:\Users\fathi\Desktop\EEG_spectrograms_npy"


# -----------------------------
# Dataset
# -----------------------------
class SpectrogramDataset(Dataset):
    def __init__(self, root_dir: str, segment: str, lobe: str, classes=(0, 1)):
        self.root_dir = root_dir
        self.segment = segment
        self.lobe = lobe
        self.classes = tuple(int(c) for c in classes)  # incoming could be list/tuple

        self.samples = []  # (path, mapped_label)
        lobe_path = os.path.join(root_dir, segment, lobe)
        if not os.path.isdir(lobe_path):
            return
        # collect .npy files for each of the desired classes
        for orig_label in self.classes:
            lbl_dir = os.path.join(lobe_path, str(orig_label))
            if not os.path.isdir(lbl_dir):
                continue
            for f in os.listdir(lbl_dir):
                if f.endswith('.npy'):
                    mapped = self.classes.index(orig_label)  # remap to 0/1
                    self.samples.append((os.path.join(lbl_dir, f), mapped))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        fpath, y = self.samples[idx]
        spec = np.load(fpath)  # (224, 224) float32 normalized
        if spec.ndim != 2:
            raise ValueError(f"Expected 2D spectrogram, got shape {spec.shape} for {fpath}")
        x = torch.from_numpy(spec).unsqueeze(0).float()  # (1, H, W)
        y = torch.tensor(y, dtype=torch.long)
        return x, y


# -----------------------------
# Model
# -----------------------------
# -----------------------------
# Model (matches paper Table 2)
# -----------------------------
class EEGSpectrogramCNN(nn.Module):
    def __init__(self, num_classes=2):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(2, 2)

        self.conv2 = nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1)
        self.dropout1 = nn.Dropout(0.25)

        self.conv3 = nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1)

        self.conv4 = nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1)
        self.dropout2 = nn.Dropout(0.25)

        # After 4 conv + pooling steps on 224×224 → 14×14
        self.fc1 = nn.Linear(32 * 14 * 14, 256)
        self.dropout3 = nn.Dropout(0.50)

        self.fc2 = nn.Linear(256, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.dropout1(x)

        x = self.pool(F.relu(self.conv3(x)))
        x = self.pool(F.relu(self.conv4(x)))
        x = self.dropout2(x)

        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.dropout3(x)
        x = self.fc2(x)
        return x


# -----------------------------
# Utilities
# -----------------------------

def discover_segments_and_lobes(root_dir: str):
    """Auto-detect segment folders and lobe folders present on disk."""
    segments = []
    lobes_by_segment = {}
    for seg in sorted(os.listdir(root_dir)):
        seg_path = os.path.join(root_dir, seg)
        if not os.path.isdir(seg_path):
            continue
        # check that it contains lobe subfolders with label subfolders
        lobes = []
        for lobe in sorted(os.listdir(seg_path)):
            lobe_path = os.path.join(seg_path, lobe)
            if not os.path.isdir(lobe_path):
                continue
            # require at least one class dir
            has_class = any(os.path.isdir(os.path.join(lobe_path, str(c))) for c in [0, 1, 2])
            if has_class:
                lobes.append(lobe)
        if lobes:
            segments.append(seg)
            lobes_by_segment[seg] = lobes
    return segments, lobes_by_segment


def compute_class_weights(y_train: np.ndarray) -> torch.Tensor:
    """Balanced class weights: n_samples / (n_classes * n_samples_per_class)."""
    values, counts = np.unique(y_train, return_counts=True)
    n_samples = len(y_train)
    n_classes = len(values)
    weights = np.zeros(n_classes, dtype=np.float32)
    for v, c in zip(values, counts):
        weights[v] = n_samples / (n_classes * float(c))
    # if a class is missing (shouldn't happen in train folds), set weight=0
    weights[np.isnan(weights)] = 0.0
    return torch.tensor(weights, dtype=torch.float32)


def evaluate(model: nn.Module, loader: DataLoader, device: torch.device):
    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for xb, yb in loader:
            xb = xb.to(device)
            logits = model(xb)
            preds = logits.argmax(1).cpu().numpy().tolist()
            y_pred.extend(preds)
            y_true.extend(yb.numpy().tolist())
    acc = accuracy_score(y_true, y_pred)
    # macro treats classes equally; weighted accounts for support
    prec_m, rec_m, f1_m, _ = precision_recall_fscore_support(y_true, y_pred, average="macro", zero_division=0)
    prec_w, rec_w, f1_w, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted", zero_division=0)
    return {
        "acc": acc,
        "precision_macro": prec_m,
        "recall_macro": rec_m,
        "f1_macro": f1_m,
        "precision_weighted": prec_w,
        "recall_weighted": rec_w,
        "f1_weighted": f1_w,
    }


# -----------------------------
# Training (single fold with best-epoch selection by F1-macro)
# -----------------------------

def train_one_fold(
    dataset: SpectrogramDataset,
    train_idx: np.ndarray,
    val_idx: np.ndarray,
    device: torch.device,
    results_dir: Path,
    max_epochs: int = 20,
    batch_size: int = 32,
    lr: float = 1e-3,
    num_workers: int = 2,
):
    train_ds = Subset(dataset, train_idx)
    val_ds = Subset(dataset, val_idx)

    y_train = np.array([dataset.samples[i][1] for i in train_idx])
    class_weights = compute_class_weights(y_train).to(device)

    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True)
    val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True)

    model = EEGSpectrogramCNN(num_classes=2).to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss(weight=class_weights)

    history_rows = []
    best_f1 = -1.0
    best_epoch = -1
    best_ckpt_path = None

    for epoch in range(1, max_epochs + 1):
        model.train()
        running_loss = 0.0
        n_samples = 0
        for xb, yb in train_loader:
            xb = xb.to(device)
            yb = yb.to(device)

            optimizer.zero_grad()
            logits = model(xb)
            loss = criterion(logits, yb)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * xb.size(0)
            n_samples += xb.size(0)

        train_loss = running_loss / max(1, n_samples)

        # Validate
        val_metrics = evaluate(model, val_loader, device)
        row = {
            "epoch": epoch,
            "train_loss": train_loss,
            **val_metrics,
        }
        history_rows.append(row)

        # Select best by F1-macro
        f1 = val_metrics["f1_macro"]
        if f1 > best_f1:
            best_f1 = f1
            best_epoch = epoch
            # Save checkpoint
            results_dir.mkdir(parents=True, exist_ok=True)
            ckpt_path = results_dir / f"best_epoch{epoch}.pt"
            torch.save(
                {
                    "state_dict": model.state_dict(),
                    "epoch": epoch,
                    "metrics": val_metrics,
                    "model_kwargs": {"num_classes": 2},
                },
                ckpt_path,
            )
            best_ckpt_path = ckpt_path

    # Save per-epoch history CSV
    hist_df = pd.DataFrame(history_rows)
    hist_csv = results_dir / "history.csv"
    hist_df.to_csv(hist_csv, index=False)

    # Reload best and re-evaluate for consistency
    if best_ckpt_path is not None:
        state = torch.load(best_ckpt_path, map_location=device)
        model.load_state_dict(state["state_dict"])
    best_metrics = evaluate(model, val_loader, device)

    return {
        "best_epoch": int(best_epoch),
        "best_ckpt": str(best_ckpt_path) if best_ckpt_path else None,
        **best_metrics,
    }


# -----------------------------
# Orchestrator for all segments & lobes
# -----------------------------

def run_all(
    root_dir: str,
    results_dir: str,
    epochs: int = 20,
    batch_size: int = 32,
    lr: float = 1e-3,
    folds: int = 5,
    seed: int = 42,
    num_workers: int = 2,
):
    set_seed(seed)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    root_dir = os.path.abspath(root_dir)
    results_dir = Path(results_dir)
    results_dir.mkdir(parents=True, exist_ok=True)

    # Discover segments and lobes
    segments, lobes_by_segment = discover_segments_and_lobes(root_dir)
    if not segments:
        raise RuntimeError(f"No valid segments found under {root_dir}")

    experiments = [(0, 1), (0, 2)]

    # Summary rows will hold one row per fold per combo + an extra row for fold='mean'
    summary_rows = []

    for classes in experiments:
        exp_name = f"classes{classes[0]}_vs_{classes[1]}"
        print(f"\n================ Experiment: {exp_name} ================")
        for seg in segments:
            lobes = lobes_by_segment.get(seg, [])
            for lobe in lobes:
                # Build dataset just to inspect size and labels
                dataset = SpectrogramDataset(root_dir, seg, lobe, classes=classes)
                n_total = len(dataset)
                if n_total == 0:
                    print(f"[SKIP] No data for seg={seg}, lobe={lobe}, classes={classes}")
                    continue

                y_all = np.array([dataset.samples[i][1] for i in range(len(dataset))])
                # Check minimal class count for each class for stratified KFold
                class_counts = np.bincount(y_all, minlength=2)
                min_class = class_counts.min()
                if min_class < folds:
                    print(
                        f"[SKIP] Not enough samples per class (min={min_class}) for Stratified {folds}-fold: "
                        f"seg={seg}, lobe={lobe}, classes={classes}"
                    )
                    continue

                print(f"\n----- Segment: {seg} | Lobe: {lobe} | N={n_total} -----")

                X_idx = np.arange(n_total)
                skf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=seed)

                for fold_i, (train_idx, val_idx) in enumerate(skf.split(X_idx, y_all), start=1):
                    fold_dir = results_dir / exp_name / seg / lobe / f"fold{fold_i}"
                    metrics = train_one_fold(
                        dataset=dataset,
                        train_idx=train_idx,
                        val_idx=val_idx,
                        device=device,
                        results_dir=fold_dir,
                        max_epochs=epochs,
                        batch_size=batch_size,
                        lr=lr,
                        num_workers=num_workers,
                    )

                    summary_rows.append(
                        {
                            "experiment": exp_name,
                            "segment": seg,
                            "lobe": lobe,
                            "fold": fold_i,
                            "best_epoch": metrics["best_epoch"],
                            "acc": metrics["acc"],
                            "precision": metrics["precision_macro"],
                            "recall": metrics["recall_macro"],
                            "f1": metrics["f1_macro"],
                            "acc_w": metrics["acc"],  # same as acc
                            "precision_w": metrics["precision_weighted"],
                            "recall_w": metrics["recall_weighted"],
                            "f1_w": metrics["f1_weighted"],
                        }
                    )

                # After all folds for this (seg,lobe), add mean row
                seg_lobe_df = pd.DataFrame([r for r in summary_rows if r["experiment"]==exp_name and r["segment"]==seg and r["lobe"]==lobe and isinstance(r["fold"], int)])
                if len(seg_lobe_df) > 0:
                    mean_row = {
                        "experiment": exp_name,
                        "segment": seg,
                        "lobe": lobe,
                        "fold": "mean",
                        "best_epoch": int(np.round(seg_lobe_df["best_epoch"].mean())),
                        "acc": seg_lobe_df["acc"].mean(),
                        "precision": seg_lobe_df["precision"].mean(),
                        "recall": seg_lobe_df["recall"].mean(),
                        "f1": seg_lobe_df["f1"].mean(),
                        "acc_w": seg_lobe_df["acc_w"].mean(),
                        "precision_w": seg_lobe_df["precision_w"].mean(),
                        "recall_w": seg_lobe_df["recall_w"].mean(),
                        "f1_w": seg_lobe_df["f1_w"].mean(),
                    }
                    summary_rows.append(mean_row)

    # Save global summary
    all_df = pd.DataFrame(summary_rows)
    global_csv = results_dir / "summary_all.csv"
    all_df.to_csv(global_csv, index=False)

    # Compute best segment length and best lobe per experiment (by mean F1 across folds)
    topline_rows = []
    for exp_name in sorted(all_df["experiment"].unique()):
        df_exp = all_df[(all_df["experiment"] == exp_name) & (all_df["fold"] == "mean")]
        if len(df_exp) == 0:
            continue
        # Best (segment, lobe) combo
        best_combo = df_exp.sort_values("f1", ascending=False).iloc[0]

        # Best segment length averaged across lobes
        seg_rank = (
            df_exp.groupby("segment")["f1"].mean().sort_values(ascending=False)
        )
        best_segment = seg_rank.index[0]
        best_segment_f1 = float(seg_rank.iloc[0])

        # Best lobe averaged across segments
        lobe_rank = (
            df_exp.groupby("lobe")["f1"].mean().sort_values(ascending=False)
        )
        best_lobe = lobe_rank.index[0]
        best_lobe_f1 = float(lobe_rank.iloc[0])

        topline_rows.append(
            {
                "experiment": exp_name,
                "best_segment_overall": best_segment,
                "best_segment_f1": best_segment_f1,
                "best_lobe_overall": best_lobe,
                "best_lobe_f1": best_lobe_f1,
                "best_combo_segment": best_combo["segment"],
                "best_combo_lobe": best_combo["lobe"],
                "best_combo_f1": float(best_combo["f1"]),
            }
        )

    topline_df = pd.DataFrame(topline_rows)
    topline_csv = results_dir / "topline_summary.csv"
    topline_df.to_csv(topline_csv, index=False)

    print("\n========================================")
    print("Training complete. Key summaries saved:")
    print(f" - Global summary: {global_csv}")
    print(f" - Topline summary: {topline_csv}")
    if len(topline_df) > 0:
        print("\nTopline (per experiment):")
        print(topline_df.to_string(index=False))
    print("========================================\n")


# -----------------------------
# CLI
# -----------------------------
if __name__ == "__main__":
    import argparse, sys

    parser = argparse.ArgumentParser()
    parser.add_argument("--root_dir", type=str, required=True, help="Root folder of spectrogram .npy files")
    parser.add_argument("--results_dir", type=str, required=True, help="Output folder for checkpoints and CSVs")
    parser.add_argument("--epochs", type=int, default=20)
    parser.add_argument("--batch_size", type=int, default=32)
    parser.add_argument("--lr", type=float, default=1e-3)
    parser.add_argument("--folds", type=int, default=5)
    parser.add_argument("--seed", type=int, default=42)
    parser.add_argument("--num_workers", type=int, default=2)

    # Detect if running inside Jupyter (ipykernel)
    if any("ipykernel_launcher" in arg for arg in sys.argv):
        print("⚠️ Detected Jupyter, using fallback defaults instead of argparse.")
        class Args: pass
        args = Args()
        args.root_dir = r"C:\Users\fathi\Desktop\EEG_spectrograms_npy"
        args.results_dir = r"C:\Users\fathi\Desktop\eeg_cv_results"
        args.epochs = 50
        args.batch_size = 32
        args.lr = 1e-4
        args.folds = 5
        args.seed = 42
        args.num_workers = 0  # safer for Windows Jupyter
    else:
        args = parser.parse_args()

    run_all(
        root_dir=args.root_dir,
        results_dir=args.results_dir,
        epochs=args.epochs,
        batch_size=args.batch_size,
        lr=args.lr,
        folds=args.folds,
        seed=args.seed,
        num_workers=args.num_workers,
    )


⚠️ Detected Jupyter, using fallback defaults instead of argparse.

================ Experiment: classes0_vs_1 ================

----- Segment: 10s | Lobe: central | N=3786 -----
